In [1]:
#%run -i ~/myLibrary/init.ipynb
from scipy.special   import erfc
from scipy.integrate import cumtrapz
from math import *
from scipy.constants import c
import matplotlib.gridspec as gridspec
from scipy.optimize import fsolve
import numpy as np
import matplotlib.pyplot as pl
import pickle, gzip
import matplotlib.pyplot as plt
na = np.array
from scipy.constants import c as clight

In [36]:
# parameters
energyFB = 450.0
gammaFB = energyFB/0.938
energySB = 6500.0
gammaSB = energySB/0.938
r0 = 1.535E-18
frev=11245.5
gamma = 479.74  #450GeV
VRF = 6.0e6    # RF Voltage at flat bottom (in V) #on the 6Dec18 Stef ran the lumimod for dif. VRF values, since for 7035- there were RF voltage reduction steps from 6 to 4MV. We stayed with 4 MV ever since fill 7092.
tauSRxy_s  = np.inf    # damping times due to synchrotron radiation at FB energy (in s) for transverse plane

#for coupling
coupl_coef_FB   = 0.0015    # Linear coupling coefficient for Model in FB
delta_ts_FB = 0.025     # Unperturbed tune split in FB

#for noise
# !!! keep in mind that, due to the peakup resolution, it should be that beamsize*delta_bpm is same at FB and at FT energy !!!!!!!!!!!
theGainFB=0.1
#noise floor of the machine at FT, this should change at FB-> if noise comes from the screen vibration then delta∝field∝energy and so ...
Eratio=(energySB/energyFB)**0.25
Eratio=1.0
delta_hb1 = 3.8e-5*Eratio
delta_vb1 = 5.3e-5*Eratio
delta_hb2 = 4.4e-5*Eratio
delta_vb2 = 5.6e-5*Eratio
#noise BPM, for deltaBPM_FB=beamSize_SB*deltaBPM_SB/beamSize_FB
deltaBPM_SB_hb1 = 220.0e-5
deltaBPM_SB_vb1 = 250.0e-5
deltaBPM_SB_hb2 = 190.0e-5
deltaBPM_SB_vb2 = 210.0e-5

#sigs = 0.09
#intensity = 1.1E11
#dppFB=0.00026
#dqp_tuneFB=15.15
#dmu = dppFB*dqp_tuneFB #with no ecloud
dmu = 0.025 #with ecloud,DQ_FB= 0.295-0.27=0.025

In [37]:
print Eratio,delta_hb1

1.94950884849 7.40813362425e-05


In [26]:
#%run -i emit_model_elastic_scattering.ipynb
%run -i IBSmodel_GY.ipynb
%run -i noiseINmodel_FB.ipynb
#%run -i LumiModel_CouplNoiseBoff_2.ipynb

479.744136461 6929.63752665


In [27]:
filln = 7334
coupling=1
noiseGrowth=1

In [5]:
with gzip.open('/eos/project/l/lhc-lumimod/LuminosityFollowUp/2018/procdata/fill_%d/fill_%d_cycle.pkl.gz'%(filln, filln), 'rb') as fid:
    filln_CycleDict = pickle.load(fid)
print    filln_CycleDict['beam_2']['Injection'].keys()

with gzip.open('/eos/project/l/lhc-lumimod/LuminosityFollowUp/2018/procdata/fill_%d/fill_%d.pkl.gz'%(filln, filln), 'rb') as fid:
    dict_fill_SB = pickle.load(fid)

['at_start', 'evolution', 't_start', 'at_end', 'filled_slots', 't_end']


In [28]:
#time each bunch stayed at FB [min]
tFB_bbb1=(na(filln_CycleDict['beam_1']['Injection']['at_end']['time_meas'])-na(filln_CycleDict['beam_1']['Injection']['at_start']['time_meas']))/60.0
tFB_bbb2=(na(filln_CycleDict['beam_2']['Injection']['at_end']['time_meas'])-na(filln_CycleDict['beam_2']['Injection']['at_start']['time_meas']))/60.0
print tFB_bbb1[1],tFB_bbb2[1]

55.733330897490184 60.18333859841029


In [29]:
#!give as input to the getNoiseGrowthFB the emittance at FT!
#average emit. over all colliding bunches at start of SB
ex1_SBi =  np.nanmean(dict_fill_SB['eh_interp_coll'][1][0]*1.0e-6)
ey1_SBi =  np.nanmean(dict_fill_SB['ev_interp_coll'][1][0]*1.0e-6)
ex2_SBi =  np.nanmean(dict_fill_SB['eh_interp_coll'][2][0]*1.0e-6)
ey2_SBi =  np.nanmean(dict_fill_SB['ev_interp_coll'][2][0]*1.0e-6)
print ' ',"HB1","    ","VB1","    ","HB2","    ","VB2"
print round(ex1_SBi,8),round(ey1_SBi,8),round(ex2_SBi,8),round(ey2_SBi,8)

  HB1      VB1      HB2      VB2
1.84e-06 2.03e-06 1.85e-06 1.92e-06


In [30]:
#!find betas from BSRT for 2017 and 2018!
%run -i betasBSRT.ipynb

bx1_FB = e_dict['betaf_h'][450][1]
by1_FB = e_dict['betaf_v'][450][1]
bx2_FB = e_dict['betaf_h'][450][2]
by2_FB = e_dict['betaf_v'][450][2]

bx1_SB = e_dict['betaf_h'][6500][1]
by1_SB = e_dict['betaf_v'][6500][1]
bx2_SB = e_dict['betaf_h'][6500][2]
by2_SB = e_dict['betaf_v'][6500][2]

print '          ',"HB1"," ","VB1"," ","HB2"," ","VB2"
print 'betas FB:',bx1_FB,by1_FB,bx2_FB,by2_FB
print 'betas SB:',bx1_SB,by1_SB,bx2_SB,by2_SB

Using calibration 2018-D
           HB1   VB1   HB2   VB2
betas FB: 204.2 292.4 193.6 343.3
betas SB: 201.5 287.0 196.0 351.0


In [38]:
dict_model          = {}
dict_model_coupling = {}

for coupling in [0,1]:
    
    if coupling != 0:
        c_fact_FB   = coupl_coef_FB   
        print "~with coupling, coupl_coef=",c_fact_FB
        if noiseGrowth:
            print"adding also noise effect"
    else:
        c_fact_FB   = 0
        print "~with no coupling, coupl_coef=",c_fact_FB
        
                        
    # dict with coupling
    #if c_fact_FB != 0:
    #    dict_model_coupling = {}
        # for coupling with the evo of parameters
        #dict_model_coupling_cont = {}
    for beam_n in [1, 2]:
        dict_model['beam_{}'.format(beam_n)] = {}
        # dict with coupling
        dict_model_coupling['beam_{}'.format(beam_n)] = {}
        dict_model_coupling['Cfactor'] = c_fact_FB
        dict_model_coupling['Delta']   = delta_ts_FB                                        
                                            
        for interval in ['Injection', 'he_before_SB']:
            dict_model['beam_{}'.format(beam_n)][interval] = {}
            dict_model['beam_{}'.format(beam_n)][interval]['filled_slots'] = filln_CycleDict['beam_{}'.format(beam_n)][interval]['filled_slots']
            dict_model['beam_{}'.format(beam_n)][interval]['at_start']={}
            # dict with coupling
            if c_fact_FB != 0:
                dict_model_coupling['beam_{}'.format(beam_n)][interval] = {}
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['filled_slots'] = filln_CycleDict['beam_{}'.format(beam_n)][interval]['filled_slots']
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_start']={}
                
            for param in ['time_meas', 'emith', 'blength', 'emitv']:
                dict_model['beam_{}'.format(beam_n)][interval]['at_start'][param] = filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start'][param]
                # dict with coupling
                dex_noisee=0
                dey_noisee=0
                if c_fact_FB != 0:
                    dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_start'][param] = filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start'][param]
                    
                    #making the calculations to find the noise growth
                    ex_norm_m1=np.array(filln_CycleDict['beam_1']['Injection']['at_start']['emith'])*1.0e-06 
                    ey_norm_m1=np.array(filln_CycleDict['beam_1']['Injection']['at_start']['emitv'])*1.0e-06 
                    ex_norm_m2=np.array(filln_CycleDict['beam_2']['Injection']['at_start']['emith'])*1.0e-06 
                    ey_norm_m2=np.array(filln_CycleDict['beam_2']['Injection']['at_start']['emitv'])*1.0e-06 
                    
                    getGrowth_hb1,deltaBPM_FB_hb1= getNoiseGrowthFB(energy=gamma*0.938, emitFB=np.nanmean(ex_norm_m1), emitSB=ex1_SBi, betaFB=bx1_FB, betaSB=bx1_SB, delta=delta_hb1, deltaBPM_SB=deltaBPM_SB_hb1)
                    getGrowth_vb1,deltaBPM_FB_vb1= getNoiseGrowthFB(energy=gamma*0.938, emitFB=np.nanmean(ey_norm_m1), emitSB=ey1_SBi, betaFB=by1_FB, betaSB=by1_SB, delta=delta_vb1, deltaBPM_SB=deltaBPM_SB_vb1)
                    getGrowth_hb2,deltaBPM_FB_hb2= getNoiseGrowthFB(energy=gamma*0.938, emitFB=np.nanmean(ex_norm_m2), emitSB=ex2_SBi, betaFB=bx2_FB, betaSB=bx2_SB, delta=delta_hb2, deltaBPM_SB=deltaBPM_SB_hb2)
                    getGrowth_vb2,deltaBPM_FB_vb2= getNoiseGrowthFB(energy=gamma*0.938, emitFB=np.nanmean(ey_norm_m2), emitSB=ey2_SBi, betaFB=by2_FB, betaSB=by2_SB, delta=delta_vb2, deltaBPM_SB=deltaBPM_SB_vb2)
                    #print getGrowth_hb1,getGrowth_vb1,getGrowth_hb2,getGrowth_vb2
                    #print deltaBPM_FB_hb1,deltaBPM_FB_vb1,deltaBPM_FB_hb2,deltaBPM_FB_vb2
                    if noiseGrowth:#emittance growth due to noise, apply for both colliding and non-colliding
                    #the noise growths are assumed to be constant along the fill, because there is the balance between the variation of the intensity and the adt impact                    
                    #improve this by giving the input parameters to the noise formula, so that to get the exact growths for each case
                    #theNoiseGrowths=na([0.038,0.057,0.036,0.05])                         
                        dex_noisee1=ex_norm_m1*getGrowth_hb1/100.0*(tFB_bbb1/60.0) #tFB_bbb1 is in min
                        dey_noisee1=ey_norm_m1*getGrowth_vb1/100.0*(tFB_bbb1/60.0)
                        dex_noisee2=ex_norm_m2*getGrowth_hb2/100.0*(tFB_bbb2/60.0)
                        dey_noisee2=ey_norm_m2*getGrowth_vb2/100.0*(tFB_bbb2/60.0)      
                        #print '~',dex_noise1
                        #dex_noise1=0.003166667*1e-06;dey_noise1=0.00475*1e-06;dex_noise2=0.003*1e-06;dey_noise2=0.004166667*1e-06; #the growth in um for 5min
                        
                        #ex_IBS_norm_m1 = ex_IBS_norm_m1  +  dex_noisee1
                        #ey_IBS_norm_m1 = ey_IBS_norm_m1  +  dey_noisee1
                        #ex_IBS_norm_m2 = ex_IBS_norm_m2  +  dex_noisee2
                        #ey_IBS_norm_m2 = ey_IBS_norm_m2  +  dey_noisee2    
                    else:
                        dex_noisee1=0;
                        dey_noisee1=0;
                        dex_noisee2=0;
                        dey_noisee2=0;         
                            
                    #print 'hb1->',np.mean(dex_noisee1),deltaBPM_FB_hb1
                    #print 'vb1->',np.mean(dey_noisee1),deltaBPM_FB_vb1
                    #print 'hb2->',np.mean(dex_noisee2),deltaBPM_FB_hb2
                    #print 'vb2->',np.mean(dey_noisee2),deltaBPM_FB_vb2
            
                    if beam_n==1:
                        dex_noisee=dex_noisee1
                        dey_noisee=dey_noisee1
                    elif beam_n==2:
                        dex_noisee=dex_noisee2
                        dey_noisee=dey_noisee2
                        
                emit_h_IBS_end, IBSx, bl_IBS_end, IBSl, emit_y_IBS_end = IBSmodel(IBSON=1, gamma=gamma,
                    bunch_intensity_p = np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['intensity']),
                    ex_norm_m         = np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['emith'])*1.0e-06,
                    ey_norm_m         = np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['emitv'])*1.0e-06,
                    bl_4sigma_s       = np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['blength']),
                    VRF_V             = VRF,
                    dt_s              = (np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['time_meas'])-np.array(filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_start']['time_meas'])))
                
            # no coupling
            dict_model['beam_{}'.format(beam_n)][interval]['at_end']               = {}
            dict_model['beam_{}'.format(beam_n)][interval]['at_end']['emith']      = emit_h_IBS_end*1.0e06
            dict_model['beam_{}'.format(beam_n)][interval]['at_end']['emitv']      = emit_y_IBS_end*1.0e06
            dict_model['beam_{}'.format(beam_n)][interval]['at_end']['blength']    = bl_IBS_end
            dict_model['beam_{}'.format(beam_n)][interval]['at_end']['time_meas']  = filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['time_meas']                
              
            
            # Calculating emittances with coupling
            if c_fact_FB != 0:
                                    
                Cfactor          = c_fact_FB**2/2/(delta_ts_FB**2+c_fact_FB**2)
                dexy             = emit_h_IBS_end - emit_y_IBS_end
    
                C_emit_h_IBS_end = emit_h_IBS_end - dexy*Cfactor
                C_emit_y_IBS_end = emit_y_IBS_end + dexy*Cfactor
            
                #adding noise
                #either we call the noise here or before the coupling, it's the same thing
                
                if noiseGrowth:
                    C_emit_h_IBS_end=C_emit_h_IBS_end+dex_noisee
                    C_emit_y_IBS_end=C_emit_y_IBS_end+dey_noisee     
                    
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_end']               = {}
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_end']['emith']      = C_emit_h_IBS_end*1.0e06
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_end']['emitv']      = C_emit_y_IBS_end*1.0e06
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_end']['blength']    = bl_IBS_end
                dict_model_coupling['beam_{}'.format(beam_n)][interval]['at_end']['time_meas']  = filln_CycleDict['beam_{}'.format(beam_n)][interval]['at_end']['time_meas']    
                
            
        print 'beam', beam_n
        #print 'dex_noise:',dex_noisee,'dey_noise:',dey_noisee
          
#if c_fact_FB != 0:
with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/FlatBottom/fill_%d_model_FB_noCoupl.pkl.gz'%(filln), 'wb') as fid: #for noise dbpm k=1/2
    print "save pkl no coupling"
    pickle.dump(dict_model, fid)
    
if noiseGrowth:
    with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/FlatBottom/fill_%d_model_FB_CouplNoise2.pkl.gz'%(filln), 'wb') as fid: #for noise dbpm k=1/2
        print "save pkl with coupling & noise"
        pickle.dump(dict_model_coupling, fid)
else:
    with gzip.open('/eos/user/s/stpapado/2018/runModel_forFill/saved_pkl/FlatBottom/fill_%d_model_FB_Coupl2.pkl.gz'%(filln), 'wb') as fid: #for noise dbpm k=1/2
        print "save pkl with coupling"
        pickle.dump(dict_model_coupling, fid)

~with no coupling, coupl_coef= 0
beam 1
beam 2
~with coupling, coupl_coef= 0.0015
adding also noise effect


/usr/local/bin/start_ipykernel.py:231: RuntimeWarning: divide by zero encountered in power
/usr/local/bin/start_ipykernel.py:244: RuntimeWarning: divide by zero encountered in divide
/usr/local/bin/start_ipykernel.py:245: RuntimeWarning: invalid value encountered in multiply
/usr/local/bin/start_ipykernel.py:239: RuntimeWarning: invalid value encountered in power
/usr/local/bin/start_ipykernel.py:240: RuntimeWarning: invalid value encountered in power
/usr/local/bin/start_ipykernel.py:7: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/bin/start_ipykernel.py:8: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/usr/local/bin/start_ipykernel.py:113: RuntimeWarning: invalid value encountered in subtract


beam 1
beam 2
save pkl no coupling
save pkl with coupling & noise
